In [2]:
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras as keras

print("modules loaded succesfully 🎉")

modules loaded succesfully 🎉


In [3]:
interpreter = tf.lite.Interpreter(model_path="./model/movenet.tflite")
interpreter.allocate_tensors()

print("MoveNet model loaded 💪")

MoveNet model loaded 💪


In [4]:
def get_formated_img(path):
    dir_split = path.split('/')
    name = dir_split[len(dir_split)-1]
    img = tf.io.read_file(path)
    image = tf.compat.v1.image.decode_jpeg(img)
    image = tf.expand_dims(image, axis=0)
    image = tf.image.resize_with_pad(image, 192,192)
    input_image = tf.cast(image, dtype=tf.float32)
    print("loaded and resized image",name,"✅")
    return input_image    

In [5]:
# sandbox:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.set_tensor(input_details[0]["index"], get_formated_img(path="./data/train/tree/guy2_tree142.jpg"))
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]["index"])
output
output = np.reshape(a=output, newshape=(17,3))
output
# np.savetxt("random.csv",output,delimiter=",")

loaded and resized image guy2_tree142.jpg ✅


array([[0.2772471 , 0.56881684, 0.55103064],
       [0.26742756, 0.5816419 , 0.54881746],
       [0.26837772, 0.5573474 , 0.53581977],
       [0.28837165, 0.5962076 , 0.42198175],
       [0.28725553, 0.53781116, 0.49413472],
       [0.34925705, 0.61476105, 0.58458763],
       [0.3452422 , 0.51171374, 0.3930148 ],
       [0.2277348 , 0.6232971 , 0.42909065],
       [0.22380474, 0.5156735 , 0.3493143 ],
       [0.11398795, 0.58409923, 0.37271482],
       [0.11256083, 0.5580526 , 0.5908617 ],
       [0.5871861 , 0.6005497 , 0.5851234 ],
       [0.56908894, 0.52149796, 0.54909843],
       [0.74045134, 0.5838804 , 0.49967617],
       [0.634549  , 0.38257274, 0.4363025 ],
       [0.9014959 , 0.5688995 , 0.6887325 ],
       [0.6277758 , 0.4043544 , 0.33366513]], dtype=float32)

In [6]:
def get_keypoints(img):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]["index"], img)
    interpreter.invoke()
    print("formed keypoints 🎯")
    keypoints = interpreter.get_tensor(output_details[0]["index"])
    return keypoints

In [7]:
def get_posture_training_data(posture):
    combined_data = list()
    for path in os.listdir("./data/train/"+posture):
        img = get_formated_img("./data/train/"+posture+"/"+path)
        img_keypoints = get_keypoints(img)
        combined_data.append(img_keypoints)
    return combined_data
    

In [8]:
def get_posture_test_data(posture):
    combined_data = list()
    for path in os.listdir("./data/test/"+posture):
        img = get_formated_img("./data/test/"+posture+"/"+path)
        img_keypoints = get_keypoints(img)
        combined_data.append(img_keypoints)
    return combined_data

In [9]:
warrior_combined_data = get_posture_training_data("warrior")
warrior_combined_data = np.array(warrior_combined_data)
warrior_combined_data = np.reshape(warrior_combined_data, newshape=(200,17,3))

tree_combined_data = get_posture_training_data("tree")
tree_combined_data = np.array(tree_combined_data)
tree_combined_data = np.reshape(tree_combined_data, newshape=(200,17,3))

dog_combined_data = get_posture_training_data("dog")
dog_combined_data = np.array(dog_combined_data)
dog_combined_data = np.reshape(dog_combined_data, newshape=(200,17,3))

loaded and resized image girl1_warrior046.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior048.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior049.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior053.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior064.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior066.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior067.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior072.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior075.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior077.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior080.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior083.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior084.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior087.jpg ✅
formed keypoints 🎯
loaded and resized image girl1_warrior089.jpg ✅
formed keypoin

In [10]:
x = []
y = []
z = []

for i in range(200):
    x.append(0)
    y.append(1)
    z.append(2)

labels = tf.constant(np.array(x+y+z))
labels = tf.cast(labels, dtype=tf.float32)
all_data = tf.constant(np.concatenate((warrior_combined_data,tree_combined_data,dog_combined_data),axis=0))

In [11]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(17,3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [12]:
model.fit(all_data, labels, epochs=20)

Epoch 1/20
19/19 [==============================] - 0s 2ms/step - loss: 0.9941 - accuracy: 0.6350
Epoch 2/20
19/19 [==============================] - 0s 1ms/step - loss: 0.8154 - accuracy: 0.8000
Epoch 3/20
19/19 [==============================] - 0s 1ms/step - loss: 0.6969 - accuracy: 0.7883
Epoch 4/20
19/19 [==============================] - 0s 2ms/step - loss: 0.6107 - accuracy: 0.8333
Epoch 5/20
19/19 [==============================] - 0s 2ms/step - loss: 0.5529 - accuracy: 0.8350
Epoch 6/20
19/19 [==============================] - 0s 3ms/step - loss: 0.5020 - accuracy: 0.8500
Epoch 7/20
19/19 [==============================] - 0s 2ms/step - loss: 0.4597 - accuracy: 0.8800
Epoch 8/20
19/19 [==============================] - 0s 2ms/step - loss: 0.4264 - accuracy: 0.8983
Epoch 9/20
19/19 [==============================] - 0s 2ms/step - loss: 0.3978 - accuracy: 0.8933
Epoch 10/20
19/19 [==============================] - 0s 2ms/step - loss: 0.3706 - accuracy: 0.9117
Epoch 11/20
19/19 [

In [13]:
model.save("./model/posture_classifier.h5")

In [14]:
test_img = get_formated_img("./data/test/tree/guy3_tree097.jpg")
test_img_keypoints = get_keypoints(test_img)
test_img_keypoints = np.reshape(test_img_keypoints, newshape=(1,17,3))
tf.constant(test_img_keypoints)
output = model.predict((test_img_keypoints))
# np.max(output)
pose_name = {0:"warrior",1:"tree",2:"dog"}
pose_name[np.argmax(output)]

loaded and resized image guy3_tree097.jpg ✅
formed keypoints 🎯


'tree'